In [7]:
from src.cloud_functions._1_fetch_data._1_13_fetch_items_promotions.main import fetch_promotions_data
import pandas as pd

In [2]:
class MockRequest:
    
    def __init__(self, json_data):
        self._json_data = json_data

    def get_json(self):
        return self._json_data

In [3]:
test_data = {
 "access_token": None,
 "client_id": "4959083987776428",
 "client_secret": "Hw9wWSydd8PMvMEJewWoMvKGYMAWyKEw",
 "seller_id": 189643563,
 "store_name": "hubsmarthome"
}

mock_request = MockRequest(test_data)

In [4]:
from src.common.cloud_storage_connector import CloudStorage
from src.common.bigquery_connector import BigQueryManager
from src.common.utils import batch_process, log_process, authenticate, fetch_items_from_storage
from src.config import settings
import json
import asyncio
import aiohttp
from datetime import datetime
import time

semaphore = asyncio.Semaphore(100)  # Control the number of simultaneous requests

async def main_async(request):
    # Parsing request data
    data = request.get_json()
    client_id = data.get('client_id')
    client_secret = data.get('client_secret')
    store_name = data.get('store_name')
    seller_id = data.get('seller_id')
    access_token = data.get('access_token')
    print('** Defining authentication... **')
    
    # Authenticate (assuming this is now centralized in utils.py or a similar file)
    if not access_token:
        access_token = authenticate(client_id, client_secret)  # You can add this to a common module
    print('** Connecting to storage and BigQuery... **')

    # Initialize storage and BigQuery
    storage = CloudStorage(credentials_path=settings.PATH_SERVICE_ACCOUNT)
    #bigquery = BigQueryManager(credentials_path=settings.PATH_SERVICE_ACCOUNT)
    
    # Define paths and table names from the config
    bucket_name = settings.BUCKET_STORES
    table_management = settings.TABLE_MANAGEMENT
    destiny_table = settings.TABLE_ITEM_PROMOTION
    # Define today's date
    today_str = datetime.today().strftime('%Y-%m-%d')
    
    # Fetch item IDs from the storage bucket
    blob_items_prefix = f'{store_name}/meli/api_response/item_detail/date={today_str}/'
    items_id = fetch_items_from_storage(storage, settings.BUCKET_STORES, blob_items_prefix, 'id')
    print(f'** Items found: {len(items_id)}**')
    
    print(f'** Cleaning blob **')
    # Path for saving promotions details
    # marketplace
    blob_basic_path_marketplace = settings.BLOB_PROMOTIONS(store_name)
    date_blob_path_marketplace = f'{blob_basic_path_marketplace}date={today_str}/'
    
    # mshops
    blob_basic_path_mshops = settings.BLOB_PROMOTIONS_MSHOPS(store_name)
    date_blob_path_mshops = f'{blob_basic_path_mshops}date={today_str}/'
    
    # Clean existing files in the storage bucket
    storage.clean_blobs(bucket_name, date_blob_path_marketplace)
    storage.clean_blobs(bucket_name, date_blob_path_mshops)
    print(f'** Starting API requests for {len(items_id)} items**')
    # URL function for API
    url_marketplace = settings.URL_PROMOTIONS_MARKETPLACE
    url_mshops = settings.URL_PROMOTIONS_MSHOPS
    headers = {'Authorization': f'Bearer {access_token}'}
    
    # Batch processing the API requests
    # PROMOTIONS MELI
    # async with aiohttp.ClientSession() as session:
    #     await batch_process(session, items_id, url_marketplace, headers, bucket_name, date_blob_path_marketplace, storage, add_item_id = True, sleep_time=5)
        
    # Batch processing the API requests
    # PROMOTIONS SHOPS
    async with aiohttp.ClientSession() as session:
        await batch_process(session, items_id, url_mshops, headers, bucket_name, date_blob_path_mshops, storage, add_item_id = True, sleep_time=5)


    
    print('** Logging process in management table... **')
    # Log the process in BigQuery
    log_process(seller_id, destiny_table, today_str, table_management, processed_to_bq=False)

    return ('Success', 200)

def fetch_promotions_data(request):
    return (main_async(request))



In [5]:
result = await fetch_promotions_data(mock_request)


** Defining authentication... **
** Connecting to storage and BigQuery... **
Using local credentials from: D:/bacar/Savvi/GLM/Desenvolvimento/service_account/service_account_datalakev2.json
Reading file: hubsmarthome/meli/api_response/item_detail/date=2024-11-12/batch_0__process_time=2024-11-12T07:00:00.251473-03:00.json
Reading file: hubsmarthome/meli/api_response/item_detail/date=2024-11-12/batch_10__process_time=2024-11-12T07:00:00.156447-03:00.json
Reading file: hubsmarthome/meli/api_response/item_detail/date=2024-11-12/batch_1__process_time=2024-11-12T07:00:00.538303-03:00.json
Reading file: hubsmarthome/meli/api_response/item_detail/date=2024-11-12/batch_2__process_time=2024-11-12T07:00:00.934662-03:00.json
Reading file: hubsmarthome/meli/api_response/item_detail/date=2024-11-12/batch_3__process_time=2024-11-12T07:00:00.303732-03:00.json
Reading file: hubsmarthome/meli/api_response/item_detail/date=2024-11-12/batch_4__process_time=2024-11-12T07:00:00.810832-03:00.json
Reading fil

CancelledError: 

In [ ]:
import pandas as pd
from datetime import datetime
from src.common.cloud_storage_connector import CloudStorage
from src.common.bigquery_connector import BigQueryManager
from src.config import settings

def insert_bq_promotions(request):
    data = request.get_json()
    store_name = data.get('store_name')
    seller_id = data.get('seller_id')

    print('** Connecting to storage and BigQuery... **')
    # Initialize storage and BigQuery
    storage = CloudStorage(credentials_path=settings.PATH_SERVICE_ACCOUNT)
    bigquery = BigQueryManager(credentials_path=settings.PATH_SERVICE_ACCOUNT)

    # Define paths and table names from the config
    bucket_name = settings.BUCKET_STORES
    table_management = settings.TABLE_MANAGEMENT
    destiny_table = settings.TABLE_ITEM_PROMOTION
    blob_promotions = settings.BLOB_PROMOTIONS(store_name)
    blob_promotions_mshops = settings.BLOB_PROMOTIONS_MSHOPS(store_name)

    # Define today's date
    today_str = datetime.today().strftime('%Y-%m-%d')

    # Get dates to treat
    list_dates_to_process = bigquery.get_list_dates_to_process(seller_id, table_management, destiny_table)

    print(f'*** Starting to process dates: {len(list_dates_to_process)} dates to process ***')

    for date in list_dates_to_process:
        date_to_process = date.strftime('%Y-%m-%d')
        print(f'Processing date: {date_to_process}')

        processed_data = []

        # Channel 'marketplace' processing
        blob_prefix = blob_promotions + f'date={date_to_process}/'
        blobs = storage.list_blobs(bucket_name, blob_prefix)
        for blob in blobs:
            print(f"Reading file: {blob.name}")
            content = storage.download_json(bucket_name, blob.name)
            for json_item in content:
                if json_item:  # Ensure json_item is not None or empty
                    for item in json_item:
                        if item:
                            processed_dict = process_response(item, 'Marketplace')
                            if processed_dict:
                                processed_data.append(processed_dict)

        # Channel 'mshops' processing
        blob_prefix_mshops = blob_promotions_mshops + f'date={date_to_process}/'
        blobs = storage.list_blobs(bucket_name, blob_prefix_mshops)
        for blob in blobs:
            print(f"Reading file: {blob.name}")
            content = storage.download_json(bucket_name, blob.name)
            for json_item in content:
                if json_item:  # Ensure json_item is not None or empty
                    for item in json_item:
                        if item:
                            processed_dict = process_response(item, 'mshops')
                            if processed_dict:
                                processed_data.append(processed_dict)

        df_processed_data = pd.DataFrame(processed_data)

        if df_processed_data.empty:
            print(f'Nenhum dado processado para a data {date_to_process}, pulando inserção...')
            continue

        df_processed_data['correspondent_date'] = pd.to_datetime(date_to_process)
        df_processed_data['process_time'] = datetime.now()
        df_processed_data['seller_id'] = seller_id

        print(f'*** Finished treating all data. {df_processed_data.shape[0]} products ***')

        if not bigquery.table_exists(destiny_table):
            print(f'Table {destiny_table} does not exist. Creating table...')
            bigquery.create_table(destiny_table, df_processed_data)

        print('** Deleting existing data **')
        bigquery.delete_existing_data(destiny_table, seller_id, date_to_process)

        print('** Correcting dataframe schema **')
        df_processed_data = bigquery.match_dataframe_schema(df_processed_data, destiny_table)

        print('** Inserting data into BQ **')
        bigquery.insert_dataframe(df_processed_data, destiny_table)

        print('** Updating log table **')
        bigquery.update_logs_table(seller_id, date_to_process, destiny_table, table_management)

    return ('Success', 200)


def process_response(json_item, channel):
    # Define required fields for both channels
    required_fields_marketplace = ['item_id', 'id', 'status', 'type', 'name', 'start_date', 'finish_date']
    required_fields_mshops = ['item_id', 'id', 'status', 'type', 'name', 'target', 'buy_quantity', 'start_date', 'finish_date']

    required_fields = required_fields_marketplace if channel == "Marketplace" else required_fields_mshops

    # Check if all required fields are present in the json_item
    for field in required_fields:
        if field not in json_item or json_item[field] is None:
            # Check within "offers" list for fields like "start_date"
            if field == "start_date" and "offers" in json_item:
                start_date_found = any(offer.get("start_date") for offer in json_item["offers"])
                if not start_date_found:
                    print(f'Missing or None field "{field}" in item: {json_item}')

    # Process data based on channel type
    if channel == "Marketplace":
        data = {
            'item_id': json_item['item_id'],
            'promotion_id': json_item['id'],
            'status': json_item['status'],
            'type': json_item['type'],
            'name': json_item['name'],
            'meli_percent': json_item.get('benefits', {}).get('meli_percent'),
            'seller_percent': json_item.get('benefits', {}).get('seller_percent'),
            'start_date': json_item['start_date'] if 'start_date' in json_item else json_item['offers'][0].get('start_date'),
            'finish_date': json_item['finish_date'],
            'channel': channel,
        }

    elif channel == "mshops":
        data = {
            'item_id': json_item['item_id'],
            'promotion_id': json_item['id'],
            'status': json_item['status'],
            'type': json_item['type'],
            'name': json_item['name'],
            'target': json_item['target'],
            'buy_quantity': json_item['buy_quantity'],
            'start_date': json_item['start_date'] if 'start_date' in json_item else json_item['offers'][0].get('start_date'),
            'finish_date': json_item['finish_date'],
            'channel': channel,
        }

    return data  # Return the processed data dictionary


In [ ]:
result = insert_bq_promotions(mock_request)

In [ ]:
import pandas as pd
from datetime import datetime
from src.common.cloud_storage_connector import CloudStorage
from src.common.bigquery_connector import BigQueryManager
from src.config import settings

def insert_bq_promotions(request):
    data = request.get_json()
    store_name = data.get('store_name')
    seller_id = data.get('seller_id')

    print('** Connecting to storage and BigQuery... **')
    # Initialize storage and BigQuery
    storage = CloudStorage(credentials_path=settings.PATH_SERVICE_ACCOUNT)
    bigquery = BigQueryManager(credentials_path=settings.PATH_SERVICE_ACCOUNT)

    # Define paths and table names from the config
    bucket_name = settings.BUCKET_STORES
    table_management = settings.TABLE_MANAGEMENT
    destiny_table = settings.TABLE_EXTERNAL_ORDERS
    blob_promotions = settings.BLOB_PROMOTIONS(store_name)
    blob_promotions_mshops = settings.BLOB_PROMOTIONS_MSHOPS(store_name)

    today_str = datetime.today().strftime('%Y-%m-%d')

    
    # Caminho para o arquivo .xlsx
    file_path = 'vendas_ml.xlsx'

    # Lendo o arquivo e criando o DataFrame
    df = pd.read_excel(file_path)

    # Exibindo as primeiras linhas do DataFrame
    print(df.head())

    df_processed_data = df

    if not bigquery.table_exists(destiny_table):
        print(f'Table {destiny_table} does not exist. Creating table...')
        bigquery.create_table(destiny_table, df_processed_data)

    print('** Deleting existing data **')
    bigquery.delete_existing_data(destiny_table, seller_id, today_str)

    print('** Correcting dataframe schema **')
    df_processed_data = bigquery.match_dataframe_schema(df_processed_data, destiny_table)

    print('** Inserting data into BQ **')
    bigquery.insert_dataframe(df_processed_data, destiny_table)

    print('** Updating log table **')
    bigquery.update_logs_table(seller_id, today_str, destiny_table, table_management)

    return ('Success', 200)


def process_response(json_item, channel):
    # Define required fields for both channels
    required_fields_marketplace = ['item_id', 'id', 'status', 'type', 'name', 'start_date', 'finish_date']
    required_fields_mshops = ['item_id', 'id', 'status', 'type', 'name', 'target', 'buy_quantity', 'start_date', 'finish_date']

    required_fields = required_fields_marketplace if channel == "Marketplace" else required_fields_mshops

    # Check if all required fields are present in the json_item
    for field in required_fields:
        if field not in json_item or json_item[field] is None:
            # Check within "offers" list for fields like "start_date"
            if field == "start_date" and "offers" in json_item:
                start_date_found = any(offer.get("start_date") for offer in json_item["offers"])
                if not start_date_found:
                    print(f'Missing or None field "{field}" in item: {json_item}')

    # Process data based on channel type
    if channel == "Marketplace":
        data = {
            'item_id': json_item['item_id'],
            'promotion_id': json_item['id'],
            'status': json_item['status'],
            'type': json_item['type'],
            'name': json_item['name'],
            'meli_percent': json_item.get('benefits', {}).get('meli_percent'),
            'seller_percent': json_item.get('benefits', {}).get('seller_percent'),
            'start_date': json_item['start_date'] if 'start_date' in json_item else json_item['offers'][0].get('start_date'),
            'finish_date': json_item['finish_date'],
            'channel': channel,
        }

    elif channel == "mshops":
        data = {
            'item_id': json_item['item_id'],
            'promotion_id': json_item['id'],
            'status': json_item['status'],
            'type': json_item['type'],
            'name': json_item['name'],
            'target': json_item['target'],
            'buy_quantity': json_item['buy_quantity'],
            'start_date': json_item['start_date'] if 'start_date' in json_item else json_item['offers'][0].get('start_date'),
            'finish_date': json_item['finish_date'],
            'channel': channel,
        }

    return data  # Return the processed data dictionary


In [ ]:
mlb_list = ['MLB4780752038','MLB4305089018','MLB4183908360','MLB5139124804','MLB2617326473','MLB3844267265','MLB4830737342','MLB3737395175','MLB4780774780','MLB4600690612','MLB4780850714','MLB3801547753','MLB3775717900','MLB3474827016','MLB4185302666','MLB3849307761','MLB3811857185','MLB4187290148','MLB5009818894','MLB3812738615','MLB3498915447','MLB2982151594','MLB4183845184','MLB3414898007','MLB3627664129','MLB3812831323','MLB3893990721','MLB5131990160','MLB3865466833','MLB3891126935','MLB3821872015','MLB4568209102','MLB3659663188','MLB5000910664','MLB5057918348','MLB3844407895','MLB3816737947','MLB3326182899','MLB5139204530','MLB4305089272','MLB5149239132','MLB3817684501','MLB5139164632','MLB2758079181','MLB3522477087','MLB5057968514','MLB3619581917','MLB3816710629','MLB5093761292','MLB3752895084','MLB2737200488','MLB4305063000','MLB3299569627','MLB4830808640','MLB4525971052','MLB3844315339','MLB3817731603','MLB4350566976','MLB3891126877','MLB4568255154','MLB3809782747','MLB2668403665','MLB5139196666','MLB3844479939','MLB4080604258','MLB3164920046','MLB4808751546','MLB4507378594','MLB2737332380','MLB3806704189','MLB3370208325','MLB3289101101','MLB3532857740','MLB4155984088','MLB3844458681','MLB3444341509','MLB3871032267','MLB3839032953','MLB3886718815','MLB3716134817','MLB3844419351','MLB2737324907','MLB3865393145','MLB3620576243','MLB3817754909','MLB5057896886','MLB2822945251','MLB3369103151','MLB4507431566','MLB4776849198','MLB4976826220','MLB3569128389','MLB3326182917','MLB5058056422','MLB3865336071','MLB3502635161','MLB3204802833','MLB4507469394','MLB3752782454','MLB5071681094','MLB4999052782','MLB4765295946','MLB3840931818','MLB4800036138','MLB4681887684','MLB3816696829','MLB3627676961','MLB5000922536','MLB3204728588','MLB3758665407','MLB3817756441','MLB5077745892','MLB4811817500','MLB5010178518','MLB5058006168','MLB3403606391','MLB5101501386','MLB3752786750','MLB3634096021','MLB3459820085','MLB5131977392','MLB3886854147','MLB5138642740','MLB4988945944','MLB4389576882','MLB3823074699','MLB3865437251','MLB4183845196','MLB5057897718','MLB4527951164','MLB3817673367','MLB4759318224','MLB5051502936','MLB3574862139','MLB5106340666','MLB2668258370','MLB4153184914','MLB3589718817','MLB5093726606','MLB4364460096','MLB3812716065','MLB5116569280','MLB4670010530','MLB5136372362','MLB5057992860','MLB3564697455','MLB3457723483','MLB5115392018','MLB3635530579','MLB4988357310','MLB3869187299','MLB3867154659','MLB3326138835','MLB2617326473','MLB4974320532','MLB3649219613','MLB5057896838','MLB3659395100','MLB3809549601','MLB2617065094','MLB3885332403','MLB3812807401','MLB3892742781','MLB3618484417','MLB4158554636','MLB5103500242','MLB3391607993','MLB2888936619','MLB3886684105','MLB3750783451','MLB4830783844','MLB4977972058','MLB3737373625','MLB4234477506','MLB5057955114','MLB3204702698','MLB5139081684','MLB3817704921','MLB4958939576','MLB4637487252','MLB4237237960','MLB3775717898','MLB3326182895','MLB4074471194','MLB3204690444','MLB5094187470','MLB3887016873','MLB3757209949','MLB3636223003','MLB3752782456','MLB5094213828','MLB3737377209','MLB3504004875','MLB4826156376','MLB2728183820','MLB4282075940','MLB3801671439','MLB2221904976','MLB5009829756','MLB4831003890','MLB3790223087','MLB4811836386','MLB3847735795','MLB3521624117','MLB5094186096','MLB3568001471','MLB3501051591','MLB2631109578','MLB3402629939','MLB3750778793','MLB3844382745','MLB3326182913','MLB4996314898','MLB5057931704','MLB3878339395','MLB3204683998','MLB4188456914','MLB3636293639','MLB3775696138','MLB3627627815','MLB5009831298','MLB3844279691','MLB3878416455','MLB4183813100','MLB5094212066','MLB4759280500','MLB3870933597','MLB3844433757','MLB5057460262','MLB3758767383','MLB3816704217','MLB3775706470','MLB4169994726','MLB3732326212','MLB3860357630','MLB3694699040','MLB3882837469','MLB3838984217','MLB3544308267','MLB4525887316','MLB3872911713','MLB2636507934','MLB5139075232','MLB3882817323','MLB4559725520','MLB5000888580','MLB3165021664','MLB3758726101','MLB5000882808','MLB3551673249','MLB3299546873','MLB4512711502','MLB5057955846','MLB3886641239','MLB4988983762','MLB3819388357','MLB3746487103','MLB3816672139','MLB3892706521','MLB3750807713','MLB3338785379','MLB3546005297','MLB4800011306','MLB4733653038','MLB4984134610','MLB3839008735','MLB4471859568','MLB2059268709','MLB3812756107','MLB3812731157','MLB3620564969','MLB3326174561','MLB4183832238','MLB4568259062','MLB3297052161','MLB2059262725','MLB3737434253','MLB4988969662','MLB4075147608','MLB3865390041','MLB4559749118','MLB3784377441','MLB3627691639','MLB3490928521','MLB3659479100','MLB5057968476','MLB3730202449','MLB3530785087','MLB4808738870','MLB5128238414','MLB3775696136','MLB5128279164','MLB3659712454','MLB3189597391','MLB3204753274','MLB3659406702','MLB2910704740','MLB4546878736','MLB4509975788','MLB3817709539','MLB5093874778','MLB3656098529','MLB5009843480','MLB3759173609','MLB3844260525','MLB3619557031','MLB5058057026','MLB4217923038','MLB3469014173','MLB5058056958','MLB4993076350','MLB3204702696','MLB3811921911','MLB3855442757','MLB5139203210','MLB3560362156','MLB2728302248','MLB4305050268','MLB5126837720','MLB4548770554','MLB3898048929','MLB3816709613','MLB4722487512','MLB3865467101','MLB3656642397','MLB3816741055','MLB3817684777','MLB3844394811','MLB3383672655','MLB5138557390','MLB5058006778','MLB3816679275','MLB2727140977','MLB2053497541','MLB4988946576','MLB5057918396','MLB3817691839','MLB3844458219','MLB3659479082','MLB5093881144','MLB3865465975','MLB2106254759','MLB2610207639','MLB5151522978','MLB5149183546','MLB3811857935','MLB2059224348','MLB5000896402','MLB5148313704','MLB3844263411','MLB5057942116','MLB3233074640','MLB3812830571','MLB3812738877','MLB5058055926','MLB2737271766','MLB3857774423','MLB3779029310','MLB5093768674','MLB3534497297','MLB3326174575','MLB3326128505','MLB3746785931','MLB3400456361','MLB3810968377','MLB2728267826','MLB3811894545','MLB3858720999','MLB3825300617','MLB4978023790','MLB4507403976','MLB5000785572','MLB4681851060','MLB3326182887','MLB4819736702','MLB3347214465','MLB2822945308','MLB5057941876','MLB3819324399','MLB4780787164','MLB4969874172','MLB5106026316','MLB4507444192','MLB3812764029','MLB3204796315','MLB3844264201','MLB3817765517','MLB3690502979','MLB4183801976','MLB4183801974','MLB5093756442','MLB3775743376','MLB2880332610','MLB3847674075','MLB3729455341','MLB5149242910','MLB5110305296','MLB3556605790','MLB3764951583','MLB5139074540','MLB3326149817','MLB2880358467','MLB3817736217','MLB3326182893','MLB2668305437','MLB3304535219','MLB3788435711','MLB3752786754','MLB3873983432','MLB3812789253','MLB3844293803','MLB3890088997','MLB3502701135','MLB3659395102','MLB3643919241','MLB3567449387','MLB3844407721','MLB3844394775','MLB3164897895','MLB4607939554','MLB3817694601','MLB4998928586','MLB2822951720','MLB3394168055','MLB3486050211','MLB5139088860','MLB3819363115','MLB3812817505','MLB2133701824','MLB3327026069','MLB4188030948','MLB5094198624','MLB3844264683','MLB3394181867','MLB3857175903','MLB3869201511','MLB2059243462','MLB3898036303','MLB5057942724','MLB3620137775','MLB5057993856','MLB5057993866','MLB5057968434','MLB3326149813','MLB3609380433','MLB4187698016','MLB3847735843','MLB5057931768','MLB3865453959','MLB5093755598','MLB3757186957','MLB5058006068','MLB3812791507','MLB4525973806','MLB3865417333','MLB3775717904','MLB3402630339','MLB3758815607','MLB3812777135','MLB1844742803','MLB3620188847','MLB3886518755','MLB3830102659','MLB3746697587','MLB3867140525','MLB3620511773','MLB3536862277','MLB3711184407','MLB5132003412','MLB5094275672','MLB3865438875','MLB3502724349','MLB3502614763','MLB3636157749','MLB3502635169','MLB5093870924','MLB3844468591','MLB3326138833','MLB3805888087','MLB3655026323','MLB4993050834','MLB3458633905','MLB3674694233','MLB3716253131','MLB3815017045','MLB3502667949','MLB3898023665','MLB5139193676','MLB4187115338','MLB5009741936','MLB5139079850','MLB3757107083','MLB3360473319','MLB2888867068','MLB4945145142','MLB3326138851','MLB3817707685','MLB3812828325','MLB3690490143','MLB4581150722','MLB3360528719']

id_list = [id.strip().upper() for id in mlb_list]  # Remove espaços e coloca em maiúsculas

# Carregar o Excel em um DataFrame e especificar que a primeira linha NÃO é um cabeçalho
df = pd.read_excel("planilha2.xlsx", header=None)  # 'header=None' indica que o Excel não tem cabeçalho

# Selecionar a coluna A (primeira coluna) que contém os IDs e aplicar limpeza de espaços e maiúsculas
coluna_ids = df.iloc[:, 0].astype(str).str.strip().str.upper()

# Filtrar os IDs que estão na lista
ids_presentes = coluna_ids[coluna_ids.isin(id_list)]

# Exibir os IDs encontrados
print("IDs encontrados no Excel:")
print(ids_presentes.tolist())

IDs encontrados no Excel:
[]


In [ ]:
import os
from src.config.settings import *

path_credentials = PATH_SERVICE_ACCOUNT

if os.path.exists(path_credentials):
    print("O arquivo existe.")
else:
    print("O arquivo não foi encontrado.")


O arquivo existe.
